# Variable Definitions API demo

## Introduction

Demonstrate a selection of the operations available on the Vardef internal API. This API is intended for creation and maintenance of _Variable Definitions_ by employees at Statistics Norway.

## Get started

This demo is intended to be run on [Dapla Lab](https://lab.dapla-test.ssb.no). Currently Vardef is not deployed to production, only the test environment, so it must be used from the Dapla Lab test environment (<https://lab.dapla-test.ssb.no>). Follow these instructions to get started.

1. [Log in to Dapla Lab](https://manual.dapla.ssb.no/statistikkere/dapla-lab.html)
1. Start either the [Jupyter](https://manual.dapla.ssb.no/statistikkere/jupyter.html) or [VSCode](https://manual.dapla.ssb.no/statistikkere/vscode-python.html) service with [Data Access configured](https://manual.dapla.ssb.no/statistikkere/rstudio.html#buckets).
1. Clone this repo `git clone https://github.com/statisticsnorway/dapla-toolbelt-metadata.git`
1. Install the packages `poetry install; poetry shell`
1. Open this notebook
1. Choose the correct kernel
1. Run this notebook

## Setup

In [ ]:
import os
from typing import Any

import requests
from rich import print_json

token = os.environ.get("OIDC_TOKEN")
active_group = os.environ.get("DAPLA_GROUP_CONTEXT")

if not token:
    msg = (
        """Environment variable 'OIDC_TOKEN' not set.
If you are running on Dapla Lab, restart your service with Buckets access configured.
If you are not running on Dapla Lab, obtain the value for it by:
    1. Launch a service (Jupyterlab or VSCode) at https://lab.dapla.ssb.no
    2. Run command 'echo $OIDC_TOKEN'
    3. Run 'export OIDC_TOKEN=<value obtained from Dapla Lab>' on this machine.
"""
    )
    raise OSError(msg)

if not active_group:
    msg = ("DAPLA_GROUP_CONTEXT environment variable not set, please set it to the name of the Dapla group you wish to act as.")
    raise OSError(msg)

print(f"Your Dapla group context is: {active_group}")

# Only available from Statistics Norway's networks, requires authorization
TEST_INTERNAL_SERVER = "https://metadata.intern.test.ssb.no"

def vardef_request(path: str, method: str="GET", body: dict | None = None, params: dict | None = None) -> Any:
    """Make a request to vardef and return the json content from the response."""
    if not params:
        params = {}

    params.update({"active_group": active_group})

    response = requests.request(
        method,
        f"{TEST_INTERNAL_SERVER}{path}",
        params=params,
        headers={"Authorization": f"Bearer {token}"},
        json=body,
        timeout=10,
        )
    response.raise_for_status()
    return response.json()

## Drafts

### Create a draft

In [ ]:
body = {
    "name": {"en": "Country Background", "nb": "Landbakgrunn", "nn": "Landbakgrunn"},
    "short_name": "landbak",
    "definition": {
        "en": "Country background is the person's own, the mother's or possibly the father's country of birth. Persons without an immigrant background always have Norway as country background. In cases where the parents have different countries of birth the mother's country of birth is chosen. If neither the person nor the parents are born abroad, country background is chosen from the first person born abroad in the order mother's mother, mother's father, father's mother, father's father.",
        "nb": "For personer født i utlandet, er dette (med noen få unntak) eget fødeland. For personer født i Norge er det foreldrenes fødeland. I de tilfeller der foreldrene har ulikt fødeland, er det morens fødeland som blir valgt. Hvis ikke personen selv eller noen av foreldrene er utenlandsfødt, hentes landbakgrunn fra de første utenlandsfødte en treffer på i rekkefølgen mormor, morfar, farmor eller farfar.",
        "nn": "For personar fødd i utlandet, er dette (med nokre få unntak) eige fødeland. For personar fødd i Noreg er det fødelandet til foreldra. I dei tilfella der foreldra har ulikt fødeland, er det fødelandet til mora som blir valt. Viss ikkje personen sjølv eller nokon av foreldra er utenlandsfødt, blir henta landsbakgrunn frå dei første utenlandsfødte ein treffar på i rekkjefølgja mormor, morfar, farmor eller farfar.",
    },
    "classification_reference": "91",
    "unit_types": ["01", "02"],
    "subject_fields": ["he04"],
    "contains_sensitive_personal_information": True,
    "measurement_type": None,
    "valid_from": "2003-01-01",
    "external_reference_uri": "https://www.ssb.no/a/metadata/conceptvariable/vardok/1919/nb",
    "comment": {
        "nb": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "nn": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "en": "As of 1 January 2003, the definition was changed to also include the grandparents' country of birth.",
    },
    "related_variable_definition_uris": ["https://example.com/"],
    "contact": {
        "title": {
            "en": "Division for population statistics",
            "nb": "Seksjon for befolkningsstatistikk",
            "nn": "Seksjon for befolkningsstatistikk",
        },
        "email": "s320@ssb.no",
    },
}

print_json(data=vardef_request("/variable-definitions", "POST", body))

### Edit a draft

In [ ]:
body = {
    "related_variable_definition_uris": [
        "https://example2.com/",
        "https://example.com/",
    ],
}

print_json(data=vardef_request("/variable-definitions/JBrjGI45", "PATCH", body))

### Delete a draft

In [ ]:
print_json(data=vardef_request("/variable-definitions/JBrjGI45", "DELETE", body))

## Variable Definitions

### List all variable definitions

In [ ]:
print_json(data=vardef_request("/variable-definitions"))

### Migrate a legacy definition from Vardok to Vardef

Note: This will raise an error if the short name already exists.

In [ ]:
# Uncomment to run, choose a variable definition which is not yet migrated.
# print_json(data=vardef_request("/vardok-migration/948", "POST"))

### Get one Variable Definition

In [ ]:
print_json(data=vardef_request("/variable-definitions/KiCPNU4n"))

## Patches

### Get one Patch

In [ ]:
print_json(data=vardef_request("/variable-definitions/KiCPNU4n/patches/1"))

### Create a Patch

Only Published Variable Definitions may have Patches created on them, Draft Variable Definitions are updated in place.

Only the fields which are to be changed need to be supplied, the others are taken from the latest patch in the last Validity Period

In [ ]:
body = {
    "related_variable_definition_uris": [
        "https://example2.com/",
        "https://example.com/",
    ],
}

print_json(data=vardef_request("/variable-definitions/KiCPNU4n/patches", "POST", body))

### List Patches

In [ ]:
print_json(data=vardef_request("/variable-definitions/KiCPNU4n/patches"))

## Validity Periods

## Create a Validity Period

In [ ]:
body = {
    "valid_from": "2025-01-01",
    "definition": {
        "nb": "Ny definisjon",
        "nn": "Ny definisjon",
        "en": "New definition",
        },
    }

print_json(data=vardef_request("/variable-definitions/KiCPNU4n/validity-periods", "POST", body))

## List Validity Periods

In [ ]:
print_json(data=vardef_request("/variable-definitions/KiCPNU4n/validity-periods"))